In [ ]:
import pandas as pd
import numpy as np

##### Utility functions 

In [ ]:
import ast

def convert_lst(str):
  '''Function for converting list representation in str to list for easier joining'''
  
  x = ast.literal_eval(str)
  return x

## Data Import

### Get the topics for ALL recipes 

#### Grabbing from train df

In [ ]:
df_recipeTrain_raw = pd.read_csv('/content/drive/MyDrive/IntroML/data/train_dataset_recipewithtopic.csv')
df_recipeTrain_raw.head(3)

,recipe_id,category,name,description,combined,cat_code,topic_list
0,9014,vegetables,cheesy potato skillet stuff,this recipe came about when a friend of mine r...,cheesy potato skillet stuff came friend mine r...,9,"[3, 13, 42]"
1,9492,meat,chicken honey glazed,serve with a green salad and a your favorite r...,chicken honey glazed serve green salad favorit...,4,"[-1, 1, 3, 4, 8, 9, 27, 29, 32, 33, 39, 40, 44..."
2,2731,meat,jaegerschnitzel,(breaded veal cutlet with mushroom sauce),jaegerschnitzel breaded veal cutlet mushroom s...,4,"[-1, 5, 8, 15, 20, 22, 39, 48, 49, 51, 54]"


In [ ]:
df_recipeTrain = df_recipeTrain_raw[['recipe_id','topic_list']]
df_recipeTrain.head(2)

,recipe_id,topic_list
0,9014,"[3, 13, 42]"
1,9492,"[-1, 1, 3, 4, 8, 9, 27, 29, 32, 33, 39, 40, 44..."


#### Grabbing from test df

In [ ]:
df_recipeTest_raw = pd.read_csv('/content/drive/MyDrive/IntroML/data/test_dataset_recipewithtopic.csv')
df_recipeTest_raw.head(3)

,recipe_id,category,name,description,combined,topic_list
0,45348,pasta,basil walnut pesto,this is my favorite pesto recipe; rich but hea...,basil walnut pesto favorite pesto rich healthy...,"[7, 31, 57]"
1,330059,pasta,bulgur and chickpeas with preserved lemon vina...,this dish is loaded with texture and flavor. w...,bulgur chickpeas preserved lemon vinaigrette l...,"[3, 4, 8, 18, 23, 25, 36, 47, 52, 54, 58, 63, ..."
2,279553,soup,turkey and brown rice chilli,fantstic crock pot recipe fron betty crocker. ...,turkey brown rice chilli fantstic crock pot fr...,"[8, 15, 22, 26, 46, 48, 52, 55, 59, 62]"


In [ ]:
df_recipeTest = df_recipeTest_raw[['recipe_id','topic_list']]
df_recipeTest.head(2)

,recipe_id,topic_list
0,45348,"[7, 31, 57]"
1,330059,"[3, 4, 8, 18, 23, 25, 36, 47, 52, 54, 58, 63, ..."


#### Merging the recipes and topics from train & test into 1 df `df_recipe`

In [ ]:
df_recipe = df_recipeTrain.append(df_recipeTest)
df_recipe.drop_duplicates(subset=['recipe_id'], inplace=True)
df_recipe.reset_index(drop=True, inplace=True)
df_recipe.head(3)

,recipe_id,topic_list
0,9014,"[3, 13, 42]"
1,9492,"[-1, 1, 3, 4, 8, 9, 27, 29, 32, 33, 39, 40, 44..."
2,2731,"[-1, 5, 8, 15, 20, 22, 39, 48, 49, 51, 54]"


In [ ]:
df_recipe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64835 entries, 0 to 64834
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   recipe_id   64835 non-null  int64 
 1   topic_list  64835 non-null  object
dtypes: int64(1), object(1)
memory usage: 1013.2+ KB


### Get ratings that users left on recipes from TRAIN dataset

In [ ]:
df_train_raw = pd.read_csv('/content/drive/MyDrive/IntroML/data/train_dataset_all.csv')
df_train_raw.head(3)

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),log calories,log fat,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
0,9014,vegetables,cheesy potato skillet stuff,5,40,357.8,33.0,6.0,22.0,30.0,37.0,8.0,5.879974,3.496508,1.791759,3.091042,3.401197,3.610918,2.079442,11297,2001-06-18,What a great quick & easy meal especially for ...,4,this recipe came about when a friend of mine r...
1,9492,meat,chicken honey glazed,7,100,484.9,11.0,189.0,20.0,110.0,6.0,16.0,6.183943,2.397895,5.241747,2.995732,4.700480,1.791759,2.772589,11297,2001-06-26,So easy and tasty. To make the clean-up even...,4,serve with a green salad and a your favorite r...
2,2731,meat,jaegerschnitzel,17,35,382.2,28.0,16.0,17.0,61.0,34.0,5.0,5.945944,3.332205,2.772589,2.833213,4.110874,3.526361,1.609438,11297,2001-07-02,The man of the house really loved the sauce wi...,4,(breaded veal cutlet with mushroom sauce)


In [ ]:
df_user = df_train_raw[['recipe_id', 'user_id', 'rating']]
df_user.head(3)

,recipe_id,user_id,rating
0,9014,11297,4
1,9492,11297,4
2,2731,11297,4


In [ ]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121668 entries, 0 to 121667
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   recipe_id  121668 non-null  int64
 1   user_id    121668 non-null  int64
 2   rating     121668 non-null  int64
dtypes: int64(3)
memory usage: 2.8 MB


## Get users' preferences using only recipes from the TRAIN dataset

`preference = [list of topics]`

Get `df` by combining `df_user` with `df_recipe` to get: `recipe_id`, `topic_list`, `user_id`, `rating`

- `df` is dataframe of user's rating on a give recipe, and the corresponding recipe's topics

### Combining `df_user` with `df_recipe`

Get a combined df of recipe, user, rating given, recipe_topic

This is to map the `df_user` which contains user --rating--> recipe, and then we get the recipe's topics from `df_recipe`

This will enable us to get the user's preference of topics: the top 5 occurence of topics amongst all recipes that were rated 5

In [ ]:
df = pd.merge(df_recipe, df_user, on='recipe_id')
df.head(3)

,recipe_id,topic_list,user_id,rating
0,9014,"[3, 13, 42]",11297,4
1,9492,"[-1, 1, 3, 4, 8, 9, 27, 29, 32, 33, 39, 40, 44...",11297,4
2,2731,"[-1, 5, 8, 15, 20, 22, 39, 48, 49, 51, 54]",11297,4


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121668 entries, 0 to 121667
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   recipe_id   121668 non-null  int64 
 1   topic_list  121668 non-null  object
 2   user_id     121668 non-null  int64 
 3   rating      121668 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 4.6+ MB


### Extract user's preference

User's preference is defined as the top 5 topics from all the topics of recipes they rated 5 for

recipes rated 5 -> get topics for each recipe -> select the most common 5 topics amongst all the topics 

In [ ]:
#get the recipes where user rated 5
df_5 = df.loc[df['rating'] == 5]
df_5.head(3)

,recipe_id,topic_list,user_id,rating
10,8308,"[-1, 1, 9, 23, 27, 28, 35, 39, 40, 45, 54]",11297,5
15,8549,"[-1, 12, 21, 24, 29, 30, 31, 32, 34, 37, 38, 4...",13483,5
17,9115,"[-1, 1, 3, 4, 7, 8, 33, 40, 41, 42, 43, 46, 50]",13483,5


In [ ]:
#convert items in topic_list from str representation of list --> actual list. This is for concentanating into 1 full list
df_5['topic_list'] = df_5['topic_list'].apply(convert_lst)

#create a dataframe of user_id -> total topics from recipes where rating = 5
df_userPref = df_5.groupby('user_id').agg({'topic_list': 'sum'})
df_userPref.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,topic_list
user_id,
1535,"[-1, 8, 18, 20, 26, 29, 39, 49, 51, 54, 5, 14,..."
2310,"[3, 15, 33, 38, 41, 45, 48, 49, -1, 5, 12, 19,..."
4439,"[21, 32, 50, 13, 25, -1, 1, 9, 27, 39, 40, 54,..."


In [ ]:
def get_toppref(lst):
  '''get top 3 occurence of topics from the total topics'''
  from heapq import heappush, heappop

  uniq_pref = set(lst)
  all_pref = []
  top_pref = []

  for topic in uniq_pref:
    count = lst.count(topic)
    heappush(all_pref, (-count, topic))
  
  for i in range(3):
    try:
      top = heappop(all_pref)
      top_pref.append(top[1])
    except:
      pass
  
  return top_pref

In [ ]:
df_userPref['top_pref'] = df_userPref['topic_list'].apply(get_toppref)
df_userPref.head(3)

,topic_list,top_pref
user_id,,
1535,"[-1, 8, 18, 20, 26, 29, 39, 49, 51, 54, 5, 14,...","[0, 32, 50]"
2310,"[3, 15, 33, 38, 41, 45, 48, 49, -1, 5, 12, 19,...","[32, -1, 21]"
4439,"[21, 32, 50, 13, 25, -1, 1, 9, 27, 39, 40, 54,...","[32, 0, 50]"


## Calculate user's `prefScore` for each recipe in `df_train`

`prefScore` = cosine similarity btween a user's preference & recipe topics

We will use this `prefScore` to predict a user's preference for recipes in `df_test`. If user appears in `df_test` but we do not have his `prefScore` calculated from `df_train`, means **new user**

##### **Convert `user_pref` into `dict_userPref` -> `user_id: [top 5 preferences]`**

For easy retrieval of user's top 5 preferences

In [ ]:
df_userPref.head(3)

,topic_list,top_pref
user_id,,
1535,"[-1, 8, 18, 20, 26, 29, 39, 49, 51, 54, 5, 14,...","[0, 32, 50]"
2310,"[3, 15, 33, 38, 41, 45, 48, 49, -1, 5, 12, 19,...","[32, -1, 21]"
4439,"[21, 32, 50, 13, 25, -1, 1, 9, 27, 39, 40, 54,...","[32, 0, 50]"


In [ ]:
dict_userPref = df_userPref['top_pref'].T.to_dict()

In [ ]:
len(dict_userPref)

378

##### **Convert `df_recipe` into a `dict_recipe` -> `recipe_id: [topics]`**

For easy retrieval of recipe's `topic_list`  

In [ ]:
df_recipe.head(3)

,recipe_id,topic_list
0,9014,"[3, 13, 42]"
1,9492,"[-1, 1, 3, 4, 8, 9, 27, 29, 32, 33, 39, 40, 44..."
2,2731,"[-1, 5, 8, 15, 20, 22, 39, 48, 49, 51, 54]"


In [ ]:
dict_recipeTopic = df_recipe.set_index('recipe_id')['topic_list'].T.to_dict()

In [ ]:
for recipe_id, topics in dict_recipeTopic.items():
  dict_recipeTopic[recipe_id] = convert_lst(topics)

##### **Function for cosine similarity**

Input = userPref (list), recipe_topic (list)

cosine similarity = dot product of userPref & recipe's topics 

In [ ]:
# get the table representation in 2 lists
# output:
# features:      2  -1  22  3  (we take only user's features)
# user:          1   1  1   1
# recipe:        0   1  1   0

from numpy import dot
from numpy.linalg import norm
import math

def get_cosine_similarity(a, b):
  '''
  function that calculates the cosine similarity between user's preference & recipe's topics
  a= user's preferences (list)
  b= recipe's topics (list)
  function will take in a,b and convert them into 2 np.arrays of same length for cos-sim calculation
  '''

  user_l = []
  recipe_l = []

  for feature in a:
    user_l.append(1)
    if feature in b:
      recipe_l.append(1)
    else:
      recipe_l.append(0)

  user_a = np.asarray(user_l)
  recipe_a = np.asarray(recipe_l)
  
  cosine_similarity = dot(user_a, recipe_a)/(norm(user_a)*norm(recipe_a))
  if math.isnan(cosine_similarity):
    cosine_similarity = 0

  return cosine_similarity

### Preparing test dataset

##### **Getting the recipes (and users) we will be TESTING out on from the TEST dataset**

In [ ]:
df_test_raw = pd.read_csv('/content/drive/MyDrive/IntroML/data/test_dataset_all.csv')
df_test_raw.head(2)

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),log calories,log fat,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
0,45348,pasta,basil walnut pesto,7,12,213.8,32.0,1.0,5.0,11.0,18.0,0.0,5.365041,3.465736,0.000000,1.609438,2.397895,2.890372,0.000000,706608,2009-10-03,Just like most I tried this recipe because I n...,4,this is my favorite pesto recipe; rich but hea...
1,330059,pasta,bulgur and chickpeas with preserved lemon vina...,17,25,225.2,12.0,6.0,24.0,13.0,5.0,11.0,5.416989,2.484907,1.791759,3.178054,2.564949,1.609438,2.397895,621626,2009-10-03,DH really enjoyed this dish. I did leave out t...,5,this dish is loaded with texture and flavor. w...


In [ ]:
# we will need.... recipe_id, user_id, rating

df_test = df_test_raw[['recipe_id', 'user_id', 'rating']]
df_test.head()

,recipe_id,user_id,rating
0,45348,706608,4
1,330059,621626,5
2,279553,621626,5
3,295248,56003,4
4,237039,706608,5


In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21893 entries, 0 to 21892
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   recipe_id  21893 non-null  int64
 1   user_id    21893 non-null  int64
 2   rating     21893 non-null  int64
dtypes: int64(3)
memory usage: 513.2 KB


##### **`dict_ratingTest`: dictionary with `user_id: {recipe_id: rating}`**

For easy retrieval of the actual ratings that a user made on a specific recipe 

In [ ]:
from collections import defaultdict
import collections.abc

dict_ratingTest = defaultdict(dict)

for row in df_test.itertuples(index=False):
  user_id = row[1]
  recipe_id = row[0]
  rating = row[2]
  dict_ratingTest[user_id][recipe_id] = rating

print(dict_ratingTest)

defaultdict(<class 'dict'>, {706608: {45348: 4, 237039: 5, 159894: 5, 200296: 5, 67256: 5, 56818: 4, 324038: 5, 38067: 3, 132511: 5, 128566: 5, 70179: 5, 316328: 5, 60420: 5, 59043: 4, 84217: 5, 49387: 4, 196455: 3, 84781: 0, 59829: 0, 22908: 5, 76697: 4, 241630: 5, 84987: 4, 73062: 5, 234501: 4, 32756: 4, 280419: 5, 107440: 5, 113828: 0, 111653: 4, 354885: 4, 322833: 3, 195793: 4, 26766: 4, 63017: 5, 350894: 4, 177812: 4, 168017: 4, 61366: 4, 28025: 4, 198485: 5, 30509: 0, 49181: 5, 320209: 3, 36952: 4, 118749: 4, 87698: 5, 126789: 5, 57466: 4, 14337: 0, 81712: 4, 31804: 5, 271888: 4, 95330: 4, 15416: 0, 103227: 0, 112683: 3, 299756: 5, 107639: 4, 207764: 4, 248055: 0, 267920: 0, 155248: 5, 32369: 0, 179672: 3, 15280: 4, 252423: 5, 296811: 0, 29671: 2, 211997: 3, 26217: 0, 23775: 5, 163565: 3, 279357: 0, 212689: 0, 67433: 5, 273007: 5, 317083: 3, 205336: 3, 178899: 5, 78824: 5, 229122: 4, 140369: 5, 130019: 5, 97745: 5, 38083: 5, 85782: 4, 56117: 4, 203900: 4, 212784: 5, 250575: 4, 31

##### **Get a `set()` of users and recipes that are in test df**

In [ ]:
set_recipeTest = set(df_test['recipe_id'])

In [ ]:
set_userTest = set(df_test['user_id'])

### Generate `dict_prefScoreTest` and `dict_recipe_healthScore`

`dict_prefScoreTest` is a dictionary in the form `user_id:{recipe_id:prefScore}`

It captures the predicted preference of users w regards to a recipe

1. User's `prefScore` -> grab from `dict_userPref`
  - if user cannot be found in `dict_userPref`, deemed as new user and unable to generate ratings (because no prior knowledge of user's preferences)
  - usually for content based, user will be asked to declare a few interests, but limitations of dataset do not enable us to do so. 

2. Recipe's topics -> grab from `dict_recipeTopics`

3. Calculate prefScore: `get_cosine_similarity()`

  - prefScore > 0.5 --> predicted as user will like recipe
  - measure against actual rating --> did user really like the recipe? If rating >=4, can say user liked the recipe 





#### Get `dict_healthRank` from csv

group 1 - most healthy 
group 3 - least healthy

Rank recommendations based on their rank for healthRank predictions


In [ ]:
# get healthRanks from csv
df_healthRank_raw = pd.read_csv('/content/drive/MyDrive/IntroML/data/train_healthy.csv')
df_healthRank = df_healthRank_raw[['recipe_id', 'groups']]
df_healthRank.head(1)

,recipe_id,groups
0,780,1


In [ ]:
df_healthRank.set_index('recipe_id', inplace=True, drop=True)
df_healthRank.head(3)

,groups
recipe_id,
780,1
164,1
3651,3


In [ ]:
# convert to dict
dict_healthRank = df_healthRank['groups'].to_dict()
print(dict_healthRank)

{780: 1, 164: 1, 3651: 3, 8566: 1, 6819: 1, 2886: 2, 8770: 1, 9058: 3, 8627: 1, 9014: 1, 9492: 1, 2731: 2, 9780: 2, 10104: 1, 7705: 1, 5478: 1, 7917: 2, 8931: 1, 8769: 2, 170: 2, 10465: 1, 8661: 1, 8308: 1, 9477: 3, 10859: 2, 902: 2, 376: 1, 10757: 1, 9120: 1, 8808: 1, 9383: 3, 8549: 1, 10225: 2, 9115: 2, 9919: 3, 10722: 1, 11284: 1, 8430: 2, 10507: 1, 10122: 1, 10654: 1, 11182: 2, 7919: 2, 10180: 1, 9495: 2, 10704: 1, 8492: 2, 617: 1, 10051: 1, 693: 1, 10183: 1, 8674: 1, 11853: 1, 11913: 3, 11914: 1, 9015: 1, 11262: 1, 11915: 1, 11020: 1, 8503: 2, 12114: 2, 8534: 3, 361: 1, 10283: 1, 10074: 2, 9418: 1, 9503: 1, 5440: 3, 834: 2, 6624: 2, 11417: 1, 11299: 1, 9257: 1, 9148: 1, 9240: 3, 12415: 1, 6598: 2, 10977: 1, 10567: 1, 9064: 2, 11649: 1, 8654: 1, 10924: 1, 11200: 1, 11150: 3, 7788: 2, 11085: 2, 12644: 1, 11513: 1, 8927: 2, 10208: 3, 11420: 1, 8486: 2, 8701: 2, 12564: 3, 3620: 1, 10883: 2, 13492: 1, 13256: 2, 12842: 1, 12538: 1, 10786: 1, 12541: 2, 12567: 2, 11818: 3, 11336: 3, 12592

#### Get prefScore from `dict_prefScoreTest` and `dict_healthScore`

- `dict_prefScoreTest_noHealthScore`: `prefScore` not mulitplied by the healthScore of recipe
- `dict_prefScoreTest_healthScore`: `prefScore` divided by healthScore of recipe



In [ ]:
from collections import defaultdict
import collections.abc

In [ ]:
dict_prefScoreTest_noHealthRank = {}
dict_prefScoreTest_healthRank = {}

new_user_count = 0
nohealthRank_count = 0
total_recipes = 0

#loop through recipes in df_test and generate a prefScore for each user
for user_id in set_userTest: 
  try:
    userPref = dict_userPref[user_id] #if we have a userPref score (meaning not a new user)
    # print(f'user_id: {user_id} preference: {userPref}')
    dict_prefScoreTest_noHealthRank[user_id] = {}
    dict_prefScoreTest_healthRank[user_id] = {}

    for recipe_id in set_recipeTest:
      try:
        recipeTopic = dict_recipeTopic[recipe_id]
        # print(recipeTopic)
       
        #get baseline prefScore of user <-> feature
        prefScore = get_cosine_similarity(userPref, recipeTopic)

        #update the dictionary databases accordingly
        dict_prefScoreTest_noHealthRank[user_id].update({recipe_id: prefScore})
        if recipe_id in dict_healthRank:
          healthRank = dict_healthRank[recipe_id]

          # we divide by healthrank to put less weight on recipes with lousier ranks.
          # this way, the best healthrank scores will come first (prefScore/1 = prefScore) while the lowest healthranks will come last
          prefScore = prefScore/healthRank
          dict_prefScoreTest_healthRank[user_id].update({recipe_id: prefScore})
        else:
          healthRank = 3
          prefScore = prefScore/healthRank
          dict_prefScoreTest_healthRank[user_id].update({recipe_id: prefScore})

      except:
        print(f'cannot get topics for recipe {recipe_id}')
        pass
  except:
    new_user_count +=1 #if new user, add to new user count
    # print(f'user {user_id} is new (we do not have an existing prefScore for user)')


print(f'Total new users: {new_user_count} out of {len(dict_ratingTest)} users')
print(f'Current test database: {len(dict_ratingTest)-new_user_count} users')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide


Total new users: 0 out of 320 users
Current test database: 320 users


In [ ]:
for k, v in dict_prefScoreTest_noHealthRank.items():
  print(k, v)
  break

#format
#user_id: {recipe_id: prefScore}

20480 {131072: 0.5773502691896258, 32770: 0, 294916: 0, 163846: 0.5773502691896258, 32777: 0, 229388: 0.5773502691896258, 294925: 0.5773502691896258, 32783: 0.5773502691896258, 32784: 0.8164965809277259, 131088: 1.0000000000000002, 131090: 0.8164965809277259, 131091: 0, 262164: 1.0000000000000002, 131093: 0.8164965809277259, 294934: 0.8164965809277259, 229398: 1.0000000000000002, 196632: 0.5773502691896258, 196633: 0, 65563: 0, 262172: 0.5773502691896258, 360477: 0, 229408: 0.8164965809277259, 294945: 1.0000000000000002, 98345: 0, 65581: 1.0000000000000002, 262191: 0.8164965809277259, 327732: 1.0000000000000002, 62: 0.8164965809277259, 229440: 0, 98370: 0.8164965809277259, 32834: 1.0000000000000002, 32844: 0.5773502691896258, 229453: 0.5773502691896258, 229454: 0.5773502691896258, 360528: 1.0000000000000002, 163922: 0.5773502691896258, 131158: 0, 196695: 0, 262230: 1.0000000000000002, 295001: 0.8164965809277259, 65628: 0.5773502691896258, 360544: 1.0000000000000002, 229473: 1.000000000

In [ ]:
i = 0
for k, v in dict_prefScoreTest_healthRank.items():
  print(k, v)
  break

#format
#user_id: {recipe_id: (prefScore, healthRank}

20480 {131072: 0.5773502691896258, 32770: 0.0, 294916: 0.0, 163846: 0.5773502691896258, 32777: 0.0, 229388: 0.5773502691896258, 294925: 0.5773502691896258, 32783: 0.5773502691896258, 32784: 0.8164965809277259, 131088: 1.0000000000000002, 131090: 0.8164965809277259, 131091: 0.0, 262164: 0.3333333333333334, 131093: 0.8164965809277259, 294934: 0.2721655269759086, 229398: 0.5000000000000001, 196632: 0.2886751345948129, 196633: 0.0, 65563: 0.0, 262172: 0.2886751345948129, 360477: 0.0, 229408: 0.8164965809277259, 294945: 0.5000000000000001, 98345: 0.0, 65581: 1.0000000000000002, 262191: 0.8164965809277259, 327732: 0.5000000000000001, 62: 0.8164965809277259, 229440: 0.0, 98370: 0.8164965809277259, 32834: 1.0000000000000002, 32844: 0.1924500897298753, 229453: 0.5773502691896258, 229454: 0.2886751345948129, 360528: 0.5000000000000001, 163922: 0.5773502691896258, 131158: 0.0, 196695: 0.0, 262230: 1.0000000000000002, 295001: 0.8164965809277259, 65628: 0.5773502691896258, 360544: 1.000000000000000

## Evaluation of model

**Precision**

Precision = 1 if any(`top_n_recipes` in `all_recipes_rated>=4_by_user`) else 0 

**Recall**

recommended recipes (defined as `prefScore>0.5`) / all the recipes left rating of 4-5 on

In [ ]:
def get_top_n_recipes(n, dict_prefScoreTest, user_id):
  from collections import Counter
  
  top_rec = Counter(dict_prefScoreTest[user_id])
  topn_rec = top_rec.most_common(n)
  dict_topn_rec = {}
  for item in topn_rec:
    dict_topn_rec[item[0]] = item[1]
  return dict_topn_rec

#### Precision

In [ ]:
#function to get precision for topn recipes over total users
def get_precision(n, dict_prefScoreTest, dict_ratingTest):

  avg_precision = 0
  total_users = 0

  for user_id in dict_prefScoreTest:
    total_users += 1 
    precision = 0

    #get topn rec for user
    topn_rec = get_top_n_recipes(n, dict_prefScoreTest, user_id) #dictionary
    # print(user_id, topn_rec)
    for recipe_id, prefScore in topn_rec.items():
      if recipe_id in dict_ratingTest[user_id]:
        ratingLeft = dict_ratingTest[user_id][recipe_id]
        if ratingLeft >=4:
          precision = 1 
          # print('Rating with >=4')

    avg_precision+=precision

    # print(f'precision for user {user_id}: {precision}')

  avg_precision=avg_precision/total_users
  # print('----------')
  print(f'Avg Precision when n={n} : {avg_precision}')

  return avg_precision

In [ ]:
print(f'Avg Precision for unweighted prefScores:')

top_n = [3,5,10]
precision_score_unweighted = {}

for n in top_n:
  avg_precision = get_precision(n, dict_prefScoreTest_noHealthRank, dict_ratingTest)
  precision_score_unweighted[n] = avg_precision

Avg Precision for unweighted prefScores:
Avg Precision when n=3 : 0.03125
Avg Precision when n=5 : 0.04375
Avg Precision when n=10 : 0.075


In [ ]:
print(f'Avg Precision for weighted prefScores:')

top_n = [3,5,10]
precision_score_weighted = {}

for n in top_n:
  avg_precision = get_precision(n, dict_prefScoreTest_healthRank, dict_ratingTest)
  precision_score_weighted[n] = avg_precision

Avg Precision for weighted prefScores:
Avg Precision when n=3 : 0.03125
Avg Precision when n=5 : 0.040625
Avg Precision when n=10 : 0.06875


#### Recall


In [ ]:
def get_recall(n, dict_prefScoreTest, dict_ratingTest):
  
  avg_recall = 0
  total_users = 0


  for user_id, prefScoreInfo in dict_prefScoreTest.items():
    total_users += 1
    recall = 0
    
    #get the top_n rec for user
    topn_rec = get_top_n_recipes(n, dict_prefScoreTest, user_id) #dictionary
    # print(user_id, topn_rec)
    for recipe_id, prefScore in topn_rec.items():
    #get dict_recipes_highRating
      dict_recipes_highRating = dict_ratingTest[user_id]

      for recipe_id, rating in dict_recipes_highRating.items():
        #check if recipe_id is within the top_n rec
        if recipe_id in topn_rec:
          recall+=1
    recall=recall/n
    avg_recall+=recall

  avg_recall=avg_recall/total_users
  
  print(f'Avg recall: {avg_recall}')

  return avg_recall

In [ ]:
print(f'Avg Recall for unweighted prefScores:')

top_n = [3,5,10]
recall_unweighted = {}

for n in top_n:
  avg_recall = get_recall(n, dict_prefScoreTest_noHealthRank, dict_ratingTest)
  recall_unweighted[n] = avg_recall

Avg Recall for unweighted prefScores:
Avg recall: 0.03125
Avg recall: 0.04375
Avg recall: 0.078125


In [ ]:
print(f'Avg Recall for weighted prefScores:')

top_n = [3,5,10]
recall_weighted = {}

for n in top_n:
  avg_recall = get_recall(n, dict_prefScoreTest_healthRank, dict_ratingTest)
  recall_weighted[n] = avg_recall

Avg Recall for weighted prefScores:
Avg recall: 0.03125
Avg recall: 0.040625
Avg recall: 0.075


#### Fscore


In [ ]:
precision_score_unweighted

{3: 0.01875, 5: 0.0375, 10: 0.05625}

In [ ]:
top_n = [3,5,10]

beta_sq = 0.5 ** 2

for n in top_n:
  precision = precision_score_unweighted[n]
  recall = recall_unweighted[n]
  fscore = ((1 + beta_sq) * precision * recall) / ((beta_sq * precision) + recall)
  print(f'Fscore when n={n} : {fscore}')

Fscore when n=3 : 0.03125
Fscore when n=5 : 0.04375
Fscore when n=10 : 0.07560483870967742


In [ ]:
top_n = [3,5,10]

beta_sq = 0.5 ** 2

for n in top_n:
  precision = precision_score_weighted[n]
  recall = recall_weighted[n]
  fscore = ((1 + beta_sq) * precision * recall) / ((beta_sq * precision) + recall)
  print(f'Fscore when n={n} : {fscore}')

Fscore when n=3 : 0.03125
Fscore when n=5 : 0.04062500000000001
Fscore when n=10 : 0.06991525423728813


### (notused) Old measures

#### *old* Precision measure

More like hit rate

In [ ]:
#function to get precision for topn recipes over total users
def get_precision(n, dict_prefScoreTest, dict_ratingTest):
  '''
  calculates the precision of prefScores against actual given ratings
  '''

  from collections import Counter
  
  avg_precision = 0
  total_users = 0

  for user_id in dict_prefScoreTest:
    total_users += 1 
    #get topn rec for user
    top_rec = Counter(dict_prefScoreTest[user_id])
    topn_rec = top_rec.most_common(n)
    leftRatingCount = 0
    precision = 0
    # print(user_id, topn_rec)
    for recipe in topn_rec:
      recipe_id = recipe[0]
      prefScore = recipe[1]
      ratingLeft = dict_ratingTest[user_id][recipe_id]

      # print(f'User {user_id} left a rating of {ratingLeft} for recipe {recipe_id}')

      if ratingLeft >=4:
        leftRatingCount +=1

    precision = leftRatingCount/n
    avg_precision+=precision

    # print(f'precision for user {user_id}: {precision}')

  avg_precision=avg_precision/total_users
  # print('----------')
  print(f'Avg Precision when n={n} : {avg_precision}')

  return avg_precision

In [ ]:
top_n = [5, 10, 20]
precision_score = {}

for n in top_n:
  avg_precision = get_precision(n, dict_prefScoreTest_noHealthScore, dict_ratingTest)
  precision_score[n] = avg_precision

Avg Precision when n=5 : 0.9333333333333342
Avg Precision when n=10 : 0.9348909657320873
Avg Precision when n=20 : 0.9163551401869151


#### *old* Recall

recommended recipes (defined as prefScore>0.5) / all the recipes left rating of 4-5 for

In [ ]:
def get_recall(dict_prefScoreTest, dict_ratingTest):
  
  avg_recall = 0
  total_users = 0

  for user_id, prefScoreInfo in dict_prefScoreTest.items():
    total_users += 1
    rec_count = 0
    highRating_count = 0

    for _, prefScore in prefScoreInfo.items():
      # print(prefScore)
      if prefScore >= 0.5:
        rec_count += 1
  
    for _, rating in dict_ratingTest[user_id].items():
      # print(rating)
      if rating >=4:
        highRating_count += 1
    
    try:
      recall = rec_count/highRating_count
    except:
      #if there is division by 0, means user did not give any high ratings.
      #If that's the case, if our rec_count > 0, our recall is 0, if rec_count = 0, recall = 1
      if rec_count == 0:
        recall = 1
      else:
        recall = 0
        
    avg_recall+=recall
    # print(recall)

  avg_recall=avg_recall/total_users
  
  print(f'Avg recall: {avg_recall}')

  return avg_recall

In [ ]:
avg_recall = get_recall(dict_prefScoreTest_noHealthRank, dict_ratingTest)

Avg recall: 347.8727827299633
